<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/top.png?raw=true'>

# Installation de la SDK Azure ML

https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py

la liste de tous les modules azure : https://pypi.org/user/amlpypi/

NB : pour ceux qui utilisent linux, il faut installer dotnet avant azure en suivant le lien suivant : https://github.com/microsoft/mlopspython/issues/295

## Installation (recommandée) avec un environnement

In [1]:
# vérifier la version de conda
!conda -V

conda 4.8.5


In [2]:
# liste des environnements disponibles
!conda env list

# conda environments:
#
azureml_da3e97fcb51801118b8e80207f3e01ad     /home/lab/.azureml/envs/azureml_da3e97fcb51801118b8e80207f3e01ad
base                  *  /home/lab/anaconda3
aibs                     /home/lab/anaconda3/envs/aibs
auto-sklearn             /home/lab/anaconda3/envs/auto-sklearn
azure                    /home/lab/anaconda3/envs/azure
emotion-celeb            /home/lab/anaconda3/envs/emotion-celeb
euler-project            /home/lab/anaconda3/envs/euler-project
ml-learning              /home/lab/anaconda3/envs/ml-learning
total                    /home/lab/anaconda3/envs/total



In [ ]:
# créer un nouvel environnement "azure" avec python 3.7 (la 3.8 pose qq problème à l'heure actuelle)
!conda create -n azure python=3.7 anaconda --yes

Remove existing environment (y/[n])? 

In [ ]:
# mettre à jour cet environnement avec le fichier environment.yml du repo prepa-dp100
# (on peut normalement directement créer l'environnement depuis le fichier sans faire l'étape précédente mais cela ne marche pas sur un remote..)
!conda env update --file env/environment.yml --yes

In [ ]:
# liste des kernels disponibles
!jupyter kernelspec list

In [ ]:
# créer un kernel associé à cet environnement (utile particulièrement en remote)
!python -m ipykernel install --user --name azure

Une fois le kernel crée, on peut le séléctionner pour executer la suite du notebook dans ce kernel.

In [ ]:
# la liste des packages installés dans l'environnement
!conda list

## Installation directe avec pip

In [ ]:
#!pip install --upgrade azureml-sdk;

In [ ]:
#!pip install --upgrade azureml-train-automl;

## Vérification de la version de la SDK Azure ML

In [3]:
import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Ready to use Azure ML 1.14.0


In [ ]:
import platform,socket,re,uuid,json,psutil,logging

def getSystemInfo():
    try:
        info={}
        info['Platforme']=platform.system()
        info['Platforme-release']=platform.release()
        info['Platforme-version']=platform.version()
        info['Architecture']=platform.machine()
        info['Hostname']=socket.gethostname()
        info['IP-address']=socket.gethostbyname(socket.gethostname())
        info['MAC-address']=':'.join(re.findall('..', '%012x' % uuid.getnode()))
        info['Processeur']=platform.processor()
        info['RAM']=str(round(psutil.virtual_memory().total / (1024.0 **3)))+" Go"
        return json.dumps(info)
    except Exception as e:
        logging.exception(e)

json.loads(getSystemInfo())

# Connexion au workspace

## télécharger le fichier de config depuis le portail

Si le notebook est executé en dehors d'Azure, il faut télécharger le fichier config.json depuis le portail https://portal.azure.com/, et le mettre dans le workspace qui contient le notebook.

Si le notebook est exécuté directement depuis le workspace Azure, le fichier de config devrait déjà être là.

## ou télécharger le fichier de config en entrant directement les infos dans le notebook

In [ ]:
import os
subscription_id = os.environ.get("SUBSCRIPTION_ID", "9114a63e-9210-4e32-97ca-b7d9e8ac403d") #ID soubscription Azure
resource_group = os.environ.get("RESOURCE_GROUP", "jt-dp100-ressources") #Resource group
workspace_name = os.environ.get("WORKSPACE_NAME", "jt-dp100") #Nom workspace

In [ ]:
from azureml.core import Workspace

try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print(ws.name, "loaded")
except:
   print("Le workspace n'a pas été trouvé")

## connexion au workspace avec le fichier config

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

In [ ]:
## afficher les infos du workspace

# Ne pas oublier à la fin de l'expérience!!
(si votre travail à utilisé une instance de calcul)

<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/down.png?raw=true'>

[api azure : ComputeTarget](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computetarget?view=azure-ml-py)

[api azure : ComputeInstance](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computeinstance.computeinstance?view=azure-ml-py)

[api azure : AmlCompute](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute%28class%29?view=azure-ml-py)

In [ ]:
# stop toutes les instances de calcul
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance and compute.get_status().state != 'Stopped':
        print('try to stop compute', compute.name)
        compute.stop(show_output=True)

# Connexion au workspace

## en téléchargeant le fichier de config

Si le notebook est executé en dehors d'Azure, il faut télécharger le fichier config.json depuis le portail https://portal.azure.com/, et le mettre dans le workspace qui contient le notebook.

Si le notebook est exécuté directement depuis le workspace Azure, le fichier de config devrait déjà être là.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

## ou en entrant directement les infos dans le notebook

In [ ]:
import os
subscription_id = os.environ.get("SUBSCRIPTION_ID", "9114a63e-9210-4e32-97ca-b7d9e8ac403d") #ID soubscription Azure
resource_group = os.environ.get("RESOURCE_GROUP", "jt-dp100-ressources") #Resource group
workspace_name = os.environ.get("WORKSPACE_NAME", "jt-dp100") #Nom workspace

In [ ]:
from azureml.core import Workspace

try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print(ws.name, "loaded")
except:
   print("Le workspace n'a pas été trouvé")

## afficher les infos du workspace

In [ ]:
from azureml.core import ComputeTarget, Datastore, Dataset

print("- Compute Targets :")
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print("\t", compute.name, ':', compute.type)
    
print("- Datastores :")
for datastore_name in ws.datastores:
    datastore = Datastore.get(ws, datastore_name)
    print("\t", datastore.name, ':', datastore.datastore_type)
    
print("- Datasets :")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name)

# Ne pas oublier à la fin de l'expérience!!
(si votre travail à utilisé une instance de calcul)

<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/down.png?raw=true'>

[api azure : ComputeTarget](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computetarget?view=azure-ml-py)

[api azure : ComputeInstance](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computeinstance.computeinstance?view=azure-ml-py)

[api azure : AmlCompute](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute%28class%29?view=azure-ml-py)

In [ ]:
# stop toutes les instances de calcul
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance and compute.get_status().state != 'Stopped':
        print('try to stop compute', compute.name)
        compute.stop(show_output=True)

In [ ]:
# liste tous les compute pour vérifier qu'elles sont éteintes
for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance:
        print(compute.name, compute.get_status())

In [ ]:
import os
from azure.common.credentials import ServicePrincipalCredentials
from azure.mgmt.resource import ResourceManagementClient

subscription_id = os.environ.get(
    'AZURE_SUBSCRIPTION_ID',
    '9a159407-5de2-4a4d-a61c-89720c3bfb79') # your Azure Subscription Id

credential = ClientSecretCredential(
    tenant_id='xxxxx',
    client_id='xxxxx',
    client_secret='xxxxx'
)

client = ResourceManagementClient(subscription_id)

In [ ]:
for item in client.resource_groups.list():
    print_item(item)

In [ ]:
credentials = ServicePrincipalCredentials(
    client_id=os.environ['AZURE_CLIENT_ID'],
    secret=os.environ['AZURE_CLIENT_SECRET'],
    tenant=os.environ['AZURE_TENANT_ID']
)

In [ ]:
# If using azure-mgmt-resource < version 15 the following line produces
# a "no attribute 'signed_session'" error:
subscription = next(subscription_client.subscriptions.list())

print(subscription.subscription_id)

In [ ]:
# If using azure-mgmt-resource < version 15 the following line produces
# a "no attribute 'signed_session'" error:
subscription = next(subscription_client.subscriptions.list())

print(subscription.subscription_id)

In [14]:
# Ressources

[api azure](https://docs.microsoft.com/en-us/python/api/azureml-core)

[parcours d'apprentissage microsoft](https://docs.microsoft.com/fr-fr/learn/paths/build-ai-solutions-with-azure-ml-service/)

[le repository microsoft](https://github.com/MicrosoftDocs/mslearn-aml-labs.git)

TypeError: __init__() missing 2 required positional arguments: 'credentials' and 'subscription_id'

In [9]:
for item in client.resource_groups.list():
    print_item(item)

AttributeError: 'DefaultAzureCredential' object has no attribute 'signed_session'

In [ ]:
credentials = ServicePrincipalCredentials(
    client_id=os.environ['AZURE_CLIENT_ID'],
    secret=os.environ['AZURE_CLIENT_SECRET'],
    tenant=os.environ['AZURE_TENANT_ID']
)

In [3]:
# If using azure-mgmt-resource < version 15 the following line produces
# a "no attribute 'signed_session'" error:
subscription = next(subscription_client.subscriptions.list())

print(subscription.subscription_id)

AttributeError: 'DefaultAzureCredential' object has no attribute 'signed_session'

In [ ]:
# If using azure-mgmt-resource < version 15 the following line produces
# a "no attribute 'signed_session'" error:
subscription = next(subscription_client.subscriptions.list())

print(subscription.subscription_id)

# Ressources

[api azure](https://docs.microsoft.com/en-us/python/api/azureml-core)

[parcours d'apprentissage microsoft](https://docs.microsoft.com/fr-fr/learn/paths/build-ai-solutions-with-azure-ml-service/)

[le repository microsoft](https://github.com/MicrosoftDocs/mslearn-aml-labs.git)